In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import re
import time

In [22]:
df_update = pd.read_html('https://fbref.com/pt/comps/32/cronograma/Primeira-Liga-Resultados-e-Calendarios#sched_2022-2023_32_1', extract_links="all")
df_update = df_update[0]

df_update.columns = [col[0] for col in df_update.columns]

df_update_getSecondTuple = ['Relatório da Partida']

for column in df_update.columns:
    if column not in df_update_getSecondTuple:
        df_update[column] = df_update[column].apply(lambda x: x[0])
    else:
        df_update[column] = df_update[column].apply(lambda x: x[1])
            
df_update['goals_home'] = df_update['Resultado'].str.split('–').str[0]
df_update['goals_away'] = df_update['Resultado'].str.split('–').str[1]
df_update.drop(['Resultado','Notas'],axis=1,inplace=True)
values = ['None']
df_update = df_update[df_update['Relatório da Partida'].isna() == False]#.reset_index() 
df_update.rename({'xG': 'xG_home', '(': 'xG_away', 'Em casa': 'home', 'Visitante': 'away', 'Público': 'Assistance', 'Sem': 'Jornada'}, axis=1, inplace=True)

In [32]:
df = pd.read_csv('output_dataset_exemplo.csv')
df.set_index('Unnamed: 0', inplace=True)

for index, row in df[df['Relatório da Partida'].str.startswith('/pt/stathead/')].iterrows():
    df.loc[index] = df_update.loc[index]

for index, row in df.iterrows():
    
    if row['Relatório da Partida'].startswith('/pt/partidas/') and ('jogador1_home' not in df.columns or pd.isna(df.loc[index,'jogador1_home'])):
    
        start = time.time()

        url = row['Relatório da Partida']
        print(url)
        r = requests.get('http://fbref.com' + url)
        soup = BeautifulSoup(r.content, 'html.parser')
        tables = soup.find_all('table')
        try:
            table = tables[0]
            table2 = tables[1] 
        except Exception:
            indice = index
        
        home_players = pd.read_html(str(table))[0].drop(11)
        away_players = pd.read_html(str(table2))[0].drop(11)
        df.loc[index,'formacao_home'] = home_players.columns[0].split('(')[1][:-1]
        df.loc[index,'formacao_away'] = away_players.columns[0].split('(')[1][:-1]
        home_players = home_players.transpose().reset_index().drop('index',axis=1)
        away_players = away_players.transpose().reset_index().drop('index',axis=1)

        print(f'meio: {time.time() - start}')
        
        stats_div = soup.find('div', {'id': 'team_stats_extra'})

        # extrai o conteúdo da div
        stats_text = stats_div.text

        linhas = stats_text.split('\n')
        palavras_chave = ['Faltas', 'Escanteios', 'Cruzamentos', 'Contatos', 'Bote defensivo', 'Cortes', 'Jogadas aéreas', 'Defesas', 'Impedimentos', 'Tiro de meta', 'Cobrança de lateral', 'Bolas longas']

        lista_strings = [string for string in linhas for palavra in palavras_chave if palavra in string]

        nova_lista = []
        for string in lista_strings:
            match = re.match(r'^(\d+)([a-zA-Z\s]+)(\d+)$', string)
            if match:
                nova_lista.append([int(match.group(1)), match.group(2).strip(), int(match.group(3))])
        
        for x in nova_lista:
            df.loc[index, f"{x[1]}_home"] = x[0]
            df.loc[index, f"{x[1]}_away"] = x[2]
            
            
        cards = {
            'home_team': {
                'yellow_card': 0,
                'red_card': 0,
                'yellow_red_card': 0
            },
            'away_team': {
                'yellow_card': 0,
                'red_card': 0,
                'yellow_red_card': 0
            }
        }

        home_team_cards_div = soup.find_all('div', class_='cards')[0]
        home_team_cards_spans = home_team_cards_div.find_all('span')
        for span in home_team_cards_spans:
            cards['home_team'][span['class'][0]] += 1

        away_team_cards_div = soup.find_all('div', class_='cards')[1]
        away_team_cards_spans = away_team_cards_div.find_all('span')
        for span in away_team_cards_spans:
            cards['away_team'][span['class'][0]] += 1
        
        df.loc[index, 'yellow_card_home'] = cards['home_team']['yellow_card']
        df.loc[index, 'red_card_home'] = cards['home_team']['red_card']
        df.loc[index, 'yellow_red_card_home'] = cards['home_team']['yellow_red_card']
        df.loc[index, 'yellow_card_away'] = cards['away_team']['yellow_card']
        df.loc[index, 'red_card_away'] = cards['away_team']['red_card']
        df.loc[index, 'yellow_red_card_away'] = cards['away_team']['yellow_red_card']
        
        for i in range(len(home_players.columns) - 1):
            column_name = f'jogador{i+1}_home'
            df.loc[index, column_name] = home_players.iloc[1, i]
        for i in range(len(away_players.columns) - 1):
            column_name = f'jogador{i+1}_away'
            df.loc[index, column_name] = away_players.iloc[1, i]

        print(f'fim: {time.time() - start}')
        if time.time() - start < 2.0:
            time.sleep(2.1 - (time.time() - start))
            
df.to_csv('output_dataset_exemplo.csv')
df.head()

/pt/partidas/8d0404e6/Pacos-de-Ferreira-Famalicao-2023Abril8-Primeira-Liga
meio: 18.67648696899414
fim: 18.822927474975586
/pt/partidas/2c59fc24/Braga-Estoril-2023Abril8-Primeira-Liga
meio: 4.069724798202515
fim: 4.147726774215698
/pt/partidas/b4c2929d/Casa-Pia-Sporting-CP-2023Abril9-Primeira-Liga
meio: 11.115244626998901
fim: 11.173245668411255
/pt/partidas/a059a637/Gil-Vicente-FC-Chaves-2023Abril10-Primeira-Liga
meio: 5.113750696182251
fim: 5.1878132820129395
/pt/partidas/d1b4cffc/Famalicao-Vitoria-Guimaraes-2023Abril14-Primeira-Liga
meio: 4.728344202041626
fim: 4.785341262817383
/pt/partidas/1201d863/Estoril-Portimonense-2023Abril15-Primeira-Liga
meio: 7.303520202636719
fim: 7.379542827606201
/pt/partidas/7506f8e8/Maritimo-Pacos-de-Ferreira-2023Abril15-Primeira-Liga
meio: 7.996131896972656
fim: 8.107128620147705
/pt/partidas/7751ba61/Chaves-Benfica-2023Abril15-Primeira-Liga
meio: 10.784069061279297
fim: 10.863060235977173
/pt/partidas/8ec826c6/Porto-Santa-Clara-2023Abril15-Primeira-

,Jornada,Dia,Data,Horário,home,xG_home,xG_away,away,Assistance,Local,Árbitro,Relatório da Partida,goals_home,goals_away,formacao_home,formacao_away,Faltas_home,Faltas_away,Escanteios_home,Escanteios_away,Cruzamentos_home,Cruzamentos_away,Contatos_home,Contatos_away,Bote defensivo_home,Bote defensivo_away,Cortes_home,Cortes_away,Defesas_home,Defesas_away,Impedimentos_home,Impedimentos_away,Tiro de meta_home,Tiro de meta_away,Bolas longas_home,Bolas longas_away,yellow_card_home,red_card_home,yellow_red_card_home,yellow_card_away,red_card_away,yellow_red_card_away,jogador1_home,jogador2_home,jogador3_home,jogador4_home,jogador5_home,jogador6_home,jogador7_home,jogador8_home,jogador9_home,jogador10_home,jogador11_home,jogador12_home,jogador13_home,jogador14_home,jogador15_home,jogador16_home,jogador17_home,jogador18_home,jogador19_home,jogador1_away,jogador2_away,jogador3_away,jogador4_away,jogador5_away,jogador6_away,jogador7_away,jogador8_away,jogador9_away,jogador10_away,jogador11_away,jogador12_away,jogador13_away,jogador14_away,jogador15_away,jogador16_away,jogador17_away,jogador18_away,jogador19_away
0,1,sex,2022-08-05,20:15,Benfica,2.7,0.4,Arouca,52.617,Estádio do Sport Lisboa e Benfica,Manuel Mota,/pt/partidas/964f4501/Benfica-Arouca-2022Agost...,4,0,4-2-3-1,5-4-1,7.0,7.0,7.0,1.0,29.0,5.0,923.0,426.0,11.0,17.0,6.0,15.0,9.0,21.0,2.0,2.0,1.0,17.0,84.0,65.0,3.0,0.0,0.0,1.0,1.0,0.0,Odisseas Vlachodimos,Gilberto,Álex Grimaldo,David Neres,Enzo Fernández,João Mário,Rafa Silva,Nicolás Otamendi,Florentino Luís,Gonçalo Ramos,Morato,Helton Leite,Jan Vertonghen,Alexander Bah,Roman Yaremchuk,Chiquinho,Julian Weigl,Petar Musa,Henrique Araújo,Ignacio De Arruabarrena,Jerome Opoku,Quaresma,Arsénio Martins Lafuente Nunes,Alan Ruiz,Antony,João Othavio Basso,Oriol Busquets,Moses Yaw,Rafa Mújica,Tiago Esgaio,Emilijus Zubas,Morlaye Sylla,André Bukia,Bruninho,Bogdan Milovanov,Ismaila Soro,Mustafa Kizza,Vitinho
1,1,sáb,2022-08-06,15:30,Rio Ave,0.2,1.4,Vizela,3.038,Estádio do Rio Ave Futebol Clube,Gustavo Correia,/pt/partidas/19380804/Rio-Ave-Vizela-2022Agost...,0,1,3-4-3,4-3-3,19.0,16.0,6.0,7.0,26.0,15.0,705.0,451.0,16.0,20.0,10.0,10.0,11.0,28.0,1.0,2.0,5.0,8.0,108.0,58.0,7.0,0.0,0.0,4.0,0.0,0.0,Jhonatan,Miguel Nóbrega,Gonçalo Rodrigues,Vítor Gomes,Joca,Abdul-Aziz Yakubu,Costinha,Pedro Amaral,Aderlan Santos,Paulo Vitor,André Pereira,Magrão,Leonardo Ruíz,Amine Oudrhiri,João Ferreira,Ukra,João Graça,Renato Pantalon,Zé Manuel,Fabijan Buntić,Bruno Wilson,Anderson Jesus,Raphael Guzzo,Kiko Bondoso,Monsuru Opeyemi,Alexis Méndez,Samu,Kévin Zohi,Christian Neiva Afonso,Tomás Costa Silva,Manuel Baldé,Carlos Isaac,Diego Rosa,Osama Rashid,Mohamed Aidara,Andrés Sarmiento,Alejandro Alvarado Jr.,Nuno Gonçalo Moreira
2,1,sáb,2022-08-06,18:00,Estoril,1.8,1.5,Famalicão,1.473,Estádio António Coimbra da Mota,Nuno Almeida,/pt/partidas/94baf8c7/Estoril-Famalicao-2022Ag...,2,0,4-2-3-1,4-4-2,14.0,11.0,1.0,2.0,7.0,25.0,518.0,576.0,17.0,16.0,8.0,15.0,16.0,13.0,0.0,2.0,9.0,11.0,90.0,85.0,4.0,0.0,0.0,1.0,0.0,0.0,Daniel Leite,Bernardo Vital,Francisco Geraldes,Arthur,Tiago Gouveia,Pedro Álvaro,Mor Ndiaye,Joãozinho,Loreintz Rosier,João Carlos,Gonçalo Esteves,João Oliveira,Volnei Feltes,Rodrigo Martins,Serginho,Elias Achouri,Titouan Thomas,Gilson Tavares,Tiago Araújo,Luiz Lúcio Reis Júnior,Rúben Lima,Alexandre Penetra,André Simões,Álex Millán,Gustavo Assuncao,Junior Kadile,Rui Fonte,Dylan Batubinsika,Martin Agirregabiria,Heriberto Tavares,Ivan Zlobin,Owen Beck,Pedro Brazão,Hernán De La Fuente,Zaydou Youssouf,Jhonder Cádiz,Cláudio Silva,Théo Fonseca
3,1,sáb,2022-08-06,20:30,Porto,3.2,1.6,Marítimo,46.309,Estádio Do Dragão,Hélder Malheiro,/pt/partidas/67a5dac8/Porto-Maritimo-2022Agost...,5,1,4-1-2-1-2,4-2-3-1,13.0,11.0,4.0,4.0,9.0,16.0,630.0,473.0,16.0,16.0,11.0,9.0,11.0,22.0,1.0,2.0,10.0,12.0,57.0,59.0,3.0,0.0,0.0,2.0,0.0,0.0,Diogo Costa,Pepe,Iván Marcano,Mateus Uribe,Mehdi Taremi,Pepê,Zaidu Sanusi,Marko Grujić,Danny Loader,João Mário Lopes,Evanilson,Cláudio Ramos,Fábio Cardoso,Ga

In [23]:
df

,index,Jornada,Dia,Data,Horário,home,xG_home,xG_away,away,Assistance,Local,Árbitro,Relatório da Partida,goals_home,goals_away
0,0,1,sex,2022-08-05,20:15,Benfica,2.7,0.4,Arouca,52.617,Estádio do Sport Lisboa e Benfica,Manuel Mota,/pt/partidas/964f4501/Benfica-Arouca-2022Agost...,4,0
1,1,1,sáb,2022-08-06,15:30,Rio Ave,0.2,1.4,Vizela,3.038,Estádio do Rio Ave Futebol Clube,Gustavo Correia,/pt/partidas/19380804/Rio-Ave-Vizela-2022Agost...,0,1
2,2,1,sáb,2022-08-06,18:00,Estoril,1.8,1.5,Famalicão,1.473,Estádio António Coimbra da Mota,Nuno Almeida,/pt/partidas/94baf8c7/Estoril-Famalicao-2022Ag...,2,0
3,3,1,sáb,2022-08-06,20:30,Porto,3.2,1.6,Marítimo,46.309,Estádio Do Dragão,Hélder Malheiro,/pt/partidas/67a5dac8/Porto-Maritimo-2022Agost...,5,1
4,4,1,dom,2022-08-07,14:30,Santa Clara,0.3,1.3,Casa Pia,1.785,Estádio de São Miguel,Bruno Jose Costa,/pt/partidas/c35a0966/Santa-Clara-Casa-Pia-202...,0,0
5,5,1,dom,2022-08-07,18:00,Braga,1.4,1.8,Sporting CP,17.714,Estádio Municipal de Braga,Fábio Veríssimo,/pt/partidas/90989ec9/Braga-Sporting-CP-2022Ag...,3,3
6,6,1,dom,2022-08-07,20:30,Portimonense,1.3,0.7,Boavista,2.590,Estádio do Portimonense SC,Cláudio Ruivo Pereira,/pt/partidas/451775b5/Portimonense-Boavista-20...,0,1
7,7,1,dom,2022-08-07,20:30,Chaves,0.9,1.0,Vitória,7.526,Estádio Municipal Eng. Manuel Branco Tei...,Manuel Oliveira,/pt/partidas/813246ed/Chaves-Vitoria-Guimaraes...,0,1
8,8,1,seg,2022-08-08,20:15,Gil Vicente FC,1.4,0.9,Paços,4.871,Estádio Cidade de Barcelos,André Narciso,/pt/partidas/c4c62ffe/Gil-Vicente-FC-Pacos-de-...,1,0
9,10,2,sex,2022-08-12,20:15,Famalicão,1.0,2.3,Braga,4.637,Estádio Municipal 22 de Junho,Luís Godinho,/pt/partidas/116f625b/Famalicao-Braga-2022Agos...,0,3
